# Entorno de experimentación `POS Less`

En este entorno se ignoran las etiquetas *POS* en la construcción de las *feature lists*

### Parámetros generales

* Maximo Iteraciones = 50
* K = 3

### Parametros por modelo

* `POSLess_reg.crfsuite`
    * l1 = 0.1
    * l2 = 0.001
* `POSLess_noreg.crfsuite`
    * l1 = 0
    * l2 = 0
* `POSLess_l1_zero.crfsuite`
    * l1 = 0
    * l2 = 0.001
* `POSLess_l2_zero.crfsuite`
    * l1 = 0.1
    * l2 = 0



## Importando bibliotecas de python

In [13]:
import os
import sys  
import random
import time
import pycrfsuite
import numpy as np
from sklearn.model_selection import KFold
from utils import (get_corpus, WordsToLetter, accuracy_score, model_trainer,
                   model_tester, write_report, eval_labeled_positions,
                   bio_classification_report)

## Funciones auxiliares    

In [14]:
def get_feature_lists(sent):
    ''' Rules that setting up the feature lists for training

    :param sent: Data as `[[[[[letter, POS, BIO-label],...],words],sents]]`
    :type: list
    :return: list of words with characters as features list:
        [[[[[letterfeatures],POS,BIO-label],letters],words]]
    :rtype: list
    '''
    featurelist = []
    senlen = len(sent)
    # each word in a sentence
    for i in range(senlen):
        word = sent[i]
        wordlen = len(word)
        lettersequence = ''
        # each letter in a word
        for j in range(wordlen):
            letter = word[j][0]
            # gathering previous letters
            lettersequence += letter
            # ignore digits
            if not letter.isdigit():
                features = [
                    'bias',
                    'letterLowercase=' + letter.lower(),
                ]
                # Position of word in sentence
                if i == senlen -1:
                    features.append("EOS")
                else:
                    features.append("BOS")

                # Position of letter in word
                if j == 0:
                    features.append('BOW')
                elif j == wordlen-1:
                    features.append('EOW')
                else:
                    features.append('letterposition=-%s' % str(wordlen-1-j))

                # Letter sequences before letter
                if j >= 4:
                    features.append('prev4letters=' + lettersequence[j-4:j].lower() + '>')
                if j >= 3:
                    features.append('prev3letters=' + lettersequence[j-3:j].lower() + '>')
                if j >= 2:
                    features.append('prev2letters=' + lettersequence[j-2:j].lower() + '>')
                if j >= 1:
                    features.append('prevletter=' + lettersequence[j-1:j].lower() + '>')

                # letter sequences after letter
                if j <= wordlen-2:
                    nxtlets = word[j+1][0]
                    features.append('nxtletter=<' + nxtlets.lower())
                if j <= wordlen-3:
                    nxtlets += word[j+2][0]
                    features.append('nxt2letters=<' + nxtlets.lower())
                if j <= wordlen-4:
                    nxtlets += word[j+3][0]
                    features.append('nxt3letters=<' + nxtlets.lower())
                if j <= wordlen-5:
                    nxtlets += word[j+4][0]
                    features.append('nxt4letters=<' + nxtlets.lower())
            featurelist.append(features)
    return featurelist


def get_labels(sent, flag=0):
    labels = []
    for word in sent:
        for letter in word:
            labels.append(letter[2])
    return labels


def sent2features(data):
    return [get_feature_lists(sent) for sent in data]


def sent2labels(data):
    return [get_labels(sent) for sent in data]

### Funciones de Train y Test 

In [15]:
def model_trainer(train_data, hyper):
    """ Entrena un modelo y lo guarda en disco

    Función encargada de entrenar un modelo con base en los hyperparametro y
    lo guarda como un archivo utilizable por `pycrfsuite`

    Parameters
    ----------
    train_data : list
    models_path : str
    hyper : dict
    verbose : bool
    k : int

    Returns
    -------
    train_time : float
        Tiempo de entrenamiento
    compositive_name : str
        Nombre del modelo entrenado
    """
    X_train = sent2features(train_data)
    y_train = sent2labels(train_data)
    
    # Train the model
    trainer = pycrfsuite.Trainer(verbose=True)

    for xseq, yseq in zip(X_train, y_train):
        trainer.append(xseq, yseq)

    # Set training parameters. L-BFGS is default. Using Elastic Net (L1 + L2)
    trainer.set_params({
            'c1': hyper['L1'],  # coefficient for L1 penalty
            'c2': hyper['L2'],  # coefficient for L2 penalty
            'max_iterations': hyper['max-iter']  # early stopping
        })
    # The program saves the trained model to a file:
    start = time.time()
    trainer.train(hyper['path'])
    end = time.time()
    train_time = end - start
    return train_time


def model_tester(test_data, model_path):
    """ Prueba un modelo preentrenado

    Recibe los datos de prueba y realiza las pruebas con el modelo previo

    Parameters
    ----------
    test_data : list
    models_path : str
    model_name : str
    verbose : bool

    Returns
    -------
    y_test : list
        Etiquetas reales
    y_pred : list
        Etiquetas predichas por el modelo
    tagger : Object
        Objeto que etiqueta con base en el modelo
    """
    X_test = sent2features(test_data)
    y_test = sent2labels(test_data)

    # ### Make Predictions
    tagger = pycrfsuite.Tagger()
    # Passing model to tagger
    tagger.open(model_path)  
    # Tagging task using the model
    y_pred = [tagger.tag(xseq) for xseq in X_test]
    # Closing tagger
    tagger.close()
    return y_test, y_pred

## Obteniendo corpus completo

In [16]:
corpus = get_corpus('corpus_otomi_mod', '../corpora/') + \
         get_corpus('corpus_otomi_hard', '../corpora/')
letter_corpus = WordsToLetter(corpus)
dataset = np.array(letter_corpus, dtype=object)

## Parametros base

In [17]:
models_path = 'models/'
env_name = "POSLess"
max_iter = 50
k = 3
kf = KFold(n_splits=k, shuffle=True)

## Parámetros para `POSLess_reg.crfsuite`

In [18]:
params = {"L1": 0.1, "L2": 1e-3, "max-iter": max_iter}
variant = "reg"

### Entrenamiento y Tests

In [19]:
%%time
i = 0
full_time = 0
accuracy_set = []
for train_index, test_index in kf.split(dataset):
    i += 1
    train_data, test_data = dataset[train_index], dataset[test_index]
    model_name = f"{env_name}_{variant}_k_{i}.crf"
    params['path'] = os.path.join(models_path, env_name, model_name)
    print("*"*50)
    print(f"Entrenando nuevo modelo '{model_name}' | K = {i}")
    print(f"len train: {len(train_data)} len test: {len(test_data)}")
    print("*"*50)
    train_time = model_trainer(train_data, params)
    full_time += train_time
    print("*"*50)
    print(f"Tiempo de entrenamiento: {train_time}[s] | {train_time / 60}[m]")
    print("Test del modelo")
    y_test, y_pred = model_tester(test_data, params['path'])
    accuracy_set.append(accuracy_score(y_test, y_pred))
    print(f"Partial accuracy: {accuracy_set[i - 1]}\n")
    # Reports
    eval_labeled_positions(y_test, y_pred)
    print(bio_classification_report(y_test, y_pred))
print("\n\nAccuracy Set -->", accuracy_set)
train_time_format = str(round(full_time / 60, 2)) + "[m]"
print(f"\nTime>> {train_time_format}")
train_size = len(train_data)
test_size = len(test_data)
params['k-folds'] = k
write_report(model_name, train_size, test_size, accuracy_set, train_time_format,
             params)

**************************************************
Entrenando nuevo modelo 'POSLess_reg_k_1.crf' | K = 1
len train: 1179 len test: 590
**************************************************
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 25566
Seconds required: 0.052

L-BFGS optimization
c1: 0.100000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 120488.360404
Feature norm: 1.000000
Error norm: 21774.691014
Active features: 25477
Line search trials: 1
Line search step: 0.000045
Seconds required for this iteration: 3.095

***** Iteration #2 *****
Loss: 94163.542014
Feature norm: 5.696220
Error norm: 26330.736423
Active features: 25422
Line search trials: 5
Line search step: 0.062500
Seconds required for this iteration: 8.025

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 26365
Seconds required: 0.046

L-BFGS optimization
c1: 0.100000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 126370.591869
Feature norm: 1.000000
Error norm: 22226.766174
Active features: 26296
Line search trials: 1
Line search step: 0.000044
Seconds required for this iteration: 4.237

***** Iteration #2 *****
Loss: 105591.527095
Feature norm: 6.334184
Error norm: 27389.902993
Active features: 26269
Line search trials: 5
Line search step: 0.062500
Seconds required for this iteration: 10.910

***** Iteration #3 *****
Loss: 64288.194994
Feature norm: 4.961873
Error norm: 7370.980003
Active features: 25400
Line search trials: 1
Line search step: 1.000000
Seconds required fo

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 26869
Seconds required: 0.047

L-BFGS optimization
c1: 0.100000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 128633.484585
Feature norm: 1.000000
Error norm: 22723.546633
Active features: 26808
Line search trials: 1
Line search step: 0.000043
Seconds required for this iteration: 3.944

***** Iteration #2 *****
Loss: 106378.318247
Feature norm: 6.140687
Error norm: 27946.029374
Active features: 26764
Line search trials: 5
Line search step: 0.062500
Seconds required for this iteration: 10.563

***** Iteration #3 *****
Loss: 66552.176598
Feature norm: 4.768346
Error norm: 6717.467811
Active features: 25879
Line search trials: 1
Line search step: 1.000000
Seconds required fo

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Parámetros para `POSLess_noreg.crfsuite`

In [20]:
params = {"L1": 0.0, "L2": 0.0, "max-iter": max_iter}
variant = "noreg"

### Entrenamiento y Tests

In [21]:
%%time
i = 0
full_time = 0
accuracy_set = []
for train_index, test_index in kf.split(dataset):
    i += 1
    train_data, test_data = dataset[train_index], dataset[test_index]
    model_name = f"{env_name}_{variant}_k_{i}.crf"
    params['path'] = os.path.join(models_path, env_name, model_name)
    print("*"*50)
    print(f"Entrenando nuevo modelo '{model_name}' | K = {i}")
    print(f"len train: {len(train_data)} len test: {len(test_data)}")
    print("*"*50)
    train_time = model_trainer(train_data, params)
    full_time += train_time
    print("*"*50)
    print(f"Tiempo de entrenamiento: {train_time}[s] | {train_time / 60}[m]")
    print("Test del modelo")
    y_test, y_pred = model_tester(test_data, params['path'])
    accuracy_set.append(accuracy_score(y_test, y_pred))
    print(f"Partial accuracy: {accuracy_set[i - 1]}\n")
    # Reports
    eval_labeled_positions(y_test, y_pred)
    print(bio_classification_report(y_test, y_pred))
print("\n\nAccuracy Set -->", accuracy_set)
train_time_format = str(round(full_time / 60, 2)) + "[m]"
print(f"\nTime>> {train_time_format}")
train_size = len(train_data)
test_size = len(test_data)
params['k-folds'] = k
write_report(model_name, train_size, test_size, accuracy_set, train_time_format,
             params)

**************************************************
Entrenando nuevo modelo 'POSLess_noreg_k_1.crf' | K = 1
len train: 1179 len test: 590
**************************************************
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 26805
Seconds required: 0.049

L-BFGS optimization
c1: 0.000000
c2: 0.000000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 92919.001155
Feature norm: 5.000000
Error norm: 25552.373816
Active features: 26805
Line search trials: 2
Line search step: 0.000216
Seconds required for this iteration: 6.154

***** Iteration #2 *****
Loss: 76068.375634
Feature norm: 3.771117
Error norm: 14221.195843
Active features: 26805
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 2.06

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 25693
Seconds required: 0.047

L-BFGS optimization
c1: 0.000000
c2: 0.000000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 89265.571021
Feature norm: 5.000000
Error norm: 24585.992703
Active features: 25693
Line search trials: 2
Line search step: 0.000226
Seconds required for this iteration: 5.483

***** Iteration #2 *****
Loss: 72750.631933
Feature norm: 3.757418
Error norm: 13527.927472
Active features: 25693
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.843

***** Iteration #3 *****
Loss: 64757.800689
Feature norm: 4.770766
Error norm: 4849.439948
Active features: 25693
Line search trials: 1
Line search step: 1.000000
Seconds required for t

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 26296
Seconds required: 0.054

L-BFGS optimization
c1: 0.000000
c2: 0.000000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 90550.552747
Feature norm: 5.000000
Error norm: 24947.202071
Active features: 26296
Line search trials: 2
Line search step: 0.000222
Seconds required for this iteration: 5.735

***** Iteration #2 *****
Loss: 73859.437429
Feature norm: 3.752277
Error norm: 13832.502442
Active features: 26296
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.828

***** Iteration #3 *****
Loss: 65673.759068
Feature norm: 4.765216
Error norm: 4916.725700
Active features: 26296
Line search trials: 1
Line search step: 1.000000
Seconds required for t

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Parámetros para `POSLess_l1_zero.crfsuite`

In [22]:
params = {"L1": 0.0, "L2": 1e-3, "max-iter": max_iter}
variant = "l1_zero"

### Entrenamiento y Tests

In [23]:
%%time
i = 0
full_time = 0
accuracy_set = []
for train_index, test_index in kf.split(dataset):
    i += 1
    train_data, test_data = dataset[train_index], dataset[test_index]
    model_name = f"{env_name}_{variant}_k_{i}.crf"
    params['path'] = os.path.join(models_path, env_name, model_name)
    print("*"*50)
    print(f"Entrenando nuevo modelo '{model_name}' | K = {i}")
    print(f"len train: {len(train_data)} len test: {len(test_data)}")
    print("*"*50)
    train_time = model_trainer(train_data, params)
    full_time += train_time
    print("*"*50)
    print(f"Tiempo de entrenamiento: {train_time}[s] | {train_time / 60}[m]")
    print("Test del modelo")
    y_test, y_pred = model_tester(test_data, params['path'])
    accuracy_set.append(accuracy_score(y_test, y_pred))
    print(f"Partial accuracy: {accuracy_set[i - 1]}\n")
    # Reports
    eval_labeled_positions(y_test, y_pred)
    print(bio_classification_report(y_test, y_pred))
print("\n\nAccuracy Set -->", accuracy_set)
train_time_format = str(round(full_time / 60, 2)) + "[m]"
print(f"\nTime>> {train_time_format}")
train_size = len(train_data)
test_size = len(test_data)
params['k-folds'] = k
write_report(model_name, train_size, test_size, accuracy_set, train_time_format,
             params)

**************************************************
Entrenando nuevo modelo 'POSLess_l1_zero_k_1.crf' | K = 1
len train: 1179 len test: 590
**************************************************
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 26659
Seconds required: 0.047

L-BFGS optimization
c1: 0.000000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 92390.706525
Feature norm: 5.000000
Error norm: 25237.206727
Active features: 26659
Line search trials: 2
Line search step: 0.000219
Seconds required for this iteration: 6.370

***** Iteration #2 *****
Loss: 76237.147753
Feature norm: 3.760312
Error norm: 14379.299905
Active features: 26659
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 2.

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 25863
Seconds required: 0.046

L-BFGS optimization
c1: 0.000000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 89179.998599
Feature norm: 5.000000
Error norm: 24804.940086
Active features: 25863
Line search trials: 2
Line search step: 0.000222
Seconds required for this iteration: 4.757

***** Iteration #2 *****
Loss: 71760.748889
Feature norm: 3.761329
Error norm: 13267.856584
Active features: 25863
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.588

***** Iteration #3 *****
Loss: 64174.081147
Feature norm: 4.757362
Error norm: 4736.711164
Active features: 25863
Line search trials: 1
Line search step: 1.000000
Seconds required for t

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 26158
Seconds required: 0.045

L-BFGS optimization
c1: 0.000000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 91168.040397
Feature norm: 5.000000
Error norm: 25054.721849
Active features: 26158
Line search trials: 2
Line search step: 0.000222
Seconds required for this iteration: 5.707

***** Iteration #2 *****
Loss: 74585.517688
Feature norm: 3.758314
Error norm: 13866.200884
Active features: 26158
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.866

***** Iteration #3 *****
Loss: 66323.186026
Feature norm: 4.777253
Error norm: 4986.759714
Active features: 26158
Line search trials: 1
Line search step: 1.000000
Seconds required for t

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Parámetros para `POSLess_l2_zero.crfsuite`

In [24]:
params = {"L1": 0.1, "L2": 0.0, "max-iter": max_iter}
variant = "l2_zero"

### Entrenamiento y Tests

In [25]:
%%time
i = 0
full_time = 0
accuracy_set = []
for train_index, test_index in kf.split(dataset):
    i += 1
    train_data, test_data = dataset[train_index], dataset[test_index]
    model_name = f"{env_name}_{variant}_k_{i}.crf"
    params['path'] = os.path.join(models_path, env_name, model_name)
    print("*"*50)
    print(f"Entrenando nuevo modelo '{model_name}' | K = {i}")
    print(f"len train: {len(train_data)} len test: {len(test_data)}")
    print("*"*50)
    train_time = model_trainer(train_data, params)
    full_time += train_time
    print("*"*50)
    print(f"Tiempo de entrenamiento: {train_time}[s] | {train_time / 60}[m]")
    print("Test del modelo")
    y_test, y_pred = model_tester(test_data, params['path'])
    accuracy_set.append(accuracy_score(y_test, y_pred))
    print(f"Partial accuracy: {accuracy_set[i - 1]}\n")
    # Reports
    eval_labeled_positions(y_test, y_pred)
    print(bio_classification_report(y_test, y_pred))
print("\n\nAccuracy Set -->", accuracy_set)
train_time_format = str(round(full_time / 60, 2)) + "[m]"
print(f"\nTime>> {train_time_format}")
train_size = len(train_data)
test_size = len(test_data)
params['k-folds'] = k
write_report(model_name, train_size, test_size, accuracy_set, train_time_format,
             params)

**************************************************
Entrenando nuevo modelo 'POSLess_l2_zero_k_1.crf' | K = 1
len train: 1179 len test: 590
**************************************************
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 25829
Seconds required: 0.049

L-BFGS optimization
c1: 0.100000
c2: 0.000000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 125034.217384
Feature norm: 1.000000
Error norm: 22408.009100
Active features: 25748
Line search trials: 1
Line search step: 0.000044
Seconds required for this iteration: 3.664

***** Iteration #2 *****
Loss: 100574.243836
Feature norm: 5.978633
Error norm: 27232.743724
Active features: 25699
Line search trials: 5
Line search step: 0.062500
Seconds required for this iteration: 

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 26843
Seconds required: 0.046

L-BFGS optimization
c1: 0.100000
c2: 0.000000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 126213.511454
Feature norm: 1.000000
Error norm: 22224.406904
Active features: 26797
Line search trials: 1
Line search step: 0.000044
Seconds required for this iteration: 3.990

***** Iteration #2 *****
Loss: 104671.584320
Feature norm: 6.153452
Error norm: 27416.188432
Active features: 26734
Line search trials: 5
Line search step: 0.062500
Seconds required for this iteration: 9.985

***** Iteration #3 *****
Loss: 65322.117833
Feature norm: 4.770051
Error norm: 6629.539517
Active features: 25867
Line search trials: 1
Line search step: 1.000000
Seconds required for

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 26243
Seconds required: 0.046

L-BFGS optimization
c1: 0.100000
c2: 0.000000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 123886.650902
Feature norm: 1.000000
Error norm: 22085.961643
Active features: 26184
Line search trials: 1
Line search step: 0.000045
Seconds required for this iteration: 3.605

***** Iteration #2 *****
Loss: 99510.758063
Feature norm: 5.960340
Error norm: 27028.205688
Active features: 26122
Line search trials: 5
Line search step: 0.062500
Seconds required for this iteration: 8.985

***** Iteration #3 *****
Loss: 63961.841051
Feature norm: 4.676852
Error norm: 5855.062974
Active features: 25258
Line search trials: 1
Line search step: 1.000000
Seconds required for 

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
